In [5]:
import glob
import geopandas as gp
import pandas as pd
import numpy as np

shpfiles=glob.glob('/home/data_folder/karamoja_shapefiles/*/*.shp')

dist_shp=gp.read_file(shpfiles[0])
#dist_shp.plot()
#dist_shp.info()

kara_shp=gp.read_file(shpfiles[1])
#kara_shp.plot()

lh_shp=gp.read_file(shpfiles[2])
#lh_shp.plot()

sc_shp=gp.read_file(shpfiles[3])
#sc_shp.plot()

dist_shp1=dist_shp[['admin2Name','geometry']]
dist_shp1.columns=['admin_name','geometry']
lh_shp1=lh_shp[['livelihood','geometry']]
lh_shp1.columns=['admin_name','geometry']
kara_shp1=kara_shp[['geometry']]
kara_shp1['admin_name']='karamoja'
#kara_shp1.columns=['admin_name','geometry']

geo_db=pd.concat([dist_shp1,lh_shp1,kara_shp1])
geo_db1=geo_db.reset_index()
geo_db2=geo_db1[['admin_name','geometry']]
geo_db2['admin_name1']=geo_db2['admin_name'].str.lower()
admin_bound_list=geo_db2['admin_name1'].tolist()
admin_bound_list

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


['abim',
 'napak',
 'nabilatuk',
 'kotido',
 'moroto',
 'nakapiripirit',
 'kaabong',
 'karenga',
 'amudat',
 'nab_nak_napak',
 'abi_kar_kaboong',
 'mor_kot_amudat',
 'karamoja']

In [74]:
lh_shp1

,admin_name,geometry
0,Nab_Nak_Napak,"POLYGON ((34.22907 1.63980, 34.22907 1.63980, ..."
1,Abi_Kar_Kaboong,"MULTIPOLYGON (((33.94424 2.63034, 33.95916 2.6..."
2,Mor_Kot_Amudat,"POLYGON ((34.27117 2.82791, 34.27038 2.81984, ..."


In [ ]:
month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}

start_month_index=2
end_month_index=9


m_ind_list=np.arange(start_month_index,end_month_index+1,1)
m_cnt=[]
for m_ind in m_ind_list:
    mstr=month_index_dict[m_ind]
    m_cnt.append(mstr)
jmstr=''.join(m_cnt)


In [45]:
import rioxarray



def get_month_index_str(start_month_index,end_month_index):
    month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
    m_ind_list=np.arange(start_month_index,end_month_index+1,1)
    m_cnt=[]
    for m_ind in m_ind_list:
        mstr=month_index_dict[m_ind]
        m_cnt.append(mstr)
    jmstr=''.join(m_cnt)
    return jmstr


def region_shp_averager_season(single_row_shp,xds,var_str,start_month_index,end_month_index):
    clipped = xds.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)
    year_xds=xds.indexes['time'].to_datetimeindex()
    year_str=str(year_xds[0].year)
    da=clipped.loc[dict(time=slice(year_str))]
    da1=da[var_str]
    aa=da1.values
    aa1=aa[start_month_index:end_month_index,:,:]
    aa2=aa1.ravel()
    aa3 = aa2[(aa2 >=-9000)]
    cont_mean=np.mean(aa3)
    return cont_mean,year_str



def make_csv_season(geo_db2,admin_bound_list,folder_str,var_str,start_month_index,end_month_index):
    ncfiles=glob.glob(f'/home/data_folder/icpac_data/data/{folder_str}/CHIRPS/*.nc')
    month_index_str=get_month_index_str(start_month_index,end_month_index)
    mdb_cont=[]
    for ncfile_path in ncfiles:
        xds = rioxarray.open_rasterio(ncfile_path)
        db_cont=[]
        year_cont=[]
        for ab_str in admin_bound_list:
            geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
            single_row_shp=geo_db3        
            cont_mean,year_str=region_shp_averager_season(single_row_shp,xds,var_str,start_month_index,end_month_index)
            year_month_index_str=f'{year_str}_{month_index_str}'
            ab_dict={}
            ab_dict['year_month']=year_month_index_str
            ab_dict['bound_name']=ab_str
            ab_dict['mean']=cont_mean
            db_cont.append(ab_dict)
        print(year_cont)
        print(db_cont)
        db=pd.DataFrame(db_cont)
        print(db)
        db1=pd.pivot_table(db, values='mean', index=['year_month'], columns=['bound_name'])
        db2=db1.reset_index()
        mdb_cont.append(db2)
        db=[]
        db1=[]
        db2=[]
    mdb1=pd.concat(mdb_cont,axis=0)
    mdb1.to_csv(f'/home/data_folder/seasonal/{folder_str}_{month_index_str}_CHRIPS.csv',index=False)
    mdb1=[]
    #return db




In [28]:
folder_list=['IGAD_Standardized_Precipitation_Index_SPI1','IGAD_Standardized_Precipitation_Index_SPI3',
            'IGAD_Standardized_Precipitation_Index_SPI6','IGAD_Standardized_Precipitation_Index_SPI9',
            'IGAD_Standardized_Precipitation_Index_SPI12','IGAD_Standardized_Precipitation_Index_SPI24',
            'IGAD_Standardized_Precipitation_Index_SPI48']

var_list=['spg01','spg03','spg06','spg09','spg12','spg24','spg48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for mar to october
start_month_index=2
end_month_index=9
    
for folder_str,var_str in zip(folder_list,var_list):
    mdb1=make_csv_season(geo_db2,admin_bound_list,folder_str,var_str,start_month_index,end_month_index)

bound_name,abi_kar_kaboong,abim,amudat,kaabong,karamoja,karenga,kotido,mor_kot_amudat,moroto,nab_nak_napak,nabilatuk,nakapiripirit,napak
year_month,,,,,,,,,,,,,
2021_MAMJJASO,-0.211082,-0.497553,0.156461,-0.129047,-0.083487,-0.040663,-0.204423,-0.04938,0.014288,0.027726,0.06378,0.212193,-0.087153


In [32]:
folder_list=['IGAD_Standardized_Precipitation_Index_SPI1','IGAD_Standardized_Precipitation_Index_SPI3',
            'IGAD_Standardized_Precipitation_Index_SPI6','IGAD_Standardized_Precipitation_Index_SPI9',
            'IGAD_Standardized_Precipitation_Index_SPI12','IGAD_Standardized_Precipitation_Index_SPI24',
            'IGAD_Standardized_Precipitation_Index_SPI48']

var_list=['spg01','spg03','spg06','spg09','spg12','spg24','spg48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for JJAS
start_month_index=5
end_month_index=8
    
for folder_str,var_str in zip(folder_list,var_list):
    mdb1=make_csv_season(geo_db2,admin_bound_list,folder_str,var_str,start_month_index,end_month_index)

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2000_JJAS', 'bound_name': 'abim', 'mean': 0.8933117}, {'year_month': '2000_JJAS', 'bound_name': 'napak', 'mean': 0.40687212}, {'year_month': '2000_JJAS', 'bound_name': 'nabilatuk', 'mean': -0.01213661}, {'year_month': '2000_JJAS', 'bound_name': 'kotido', 'mean': 0.5693305}, {'year_month': '2000_JJAS', 'bound_name': 'moroto', 'mean': -0.03919828}, {'year_month': '2000_JJAS', 'bound_name': 'nakapiripirit', 'mean': -0.07801688}, {'year_month': '2000_JJAS', 'bound_name': 'kaabong', 'mean': -0.066006534}, {'year_month': '2000_JJAS', 'bound_name': 'karenga', 'mean': 0.13393414}, {'year_month': '2000_JJAS', 'bound_name': 'amudat', 'mean': 0.04861005}, {'year_month': '2000_JJAS', 'bound_name': 'nab_nak_napak', 'mean': 0.18356526}, {'year_month': '2000_JJAS', 'bound_name': 'abi_kar_kaboong', 'mean': 0.2543998}, {'year_month': '2000_JJAS', 'bound_name': 'mor_kot_amudat', 'mean': 0.2272137}, {'year_month': '2000_JJAS', 'bound_name': 'karamoja', 'mean': 0.22318612}]
   year_mon

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2001_JJAS', 'bound_name': 'abim', 'mean': 0.6405145}, {'year_month': '2001_JJAS', 'bound_name': 'napak', 'mean': 1.1202877}, {'year_month': '2001_JJAS', 'bound_name': 'nabilatuk', 'mean': 1.1315192}, {'year_month': '2001_JJAS', 'bound_name': 'kotido', 'mean': 0.42896897}, {'year_month': '2001_JJAS', 'bound_name': 'moroto', 'mean': 0.47872415}, {'year_month': '2001_JJAS', 'bound_name': 'nakapiripirit', 'mean': 0.95673}, {'year_month': '2001_JJAS', 'bound_name': 'kaabong', 'mean': 0.16493681}, {'year_month': '2001_JJAS', 'bound_name': 'karenga', 'mean': 0.29966256}, {'year_month': '2001_JJAS', 'bound_name': 'amudat', 'mean': 0.8259057}, {'year_month': '2001_JJAS', 'bound_name': 'nab_nak_napak', 'mean': 1.0775046}, {'year_month': '2001_JJAS', 'bound_name': 'abi_kar_kaboong', 'mean': 0.33262372}, {'year_month': '2001_JJAS', 'bound_name': 'mor_kot_amudat', 'mean': 0.52238095}, {'year_month': '2001_JJAS', 'bound_name': 'karamoja', 'mean': 0.6301724}]
   year_month       b

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

...

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2020_JJAS', 'bound_name': 'abim', 'mean': 2.4405544}, {'year_month': '2020_JJAS', 'bound_name': 'napak', 'mean': 3.3399334}, {'year_month': '2020_JJAS', 'bound_name': 'nabilatuk', 'mean': 3.4450984}, {'year_month': '2020_JJAS', 'bound_name': 'kotido', 'mean': 2.8628645}, {'year_month': '2020_JJAS', 'bound_name': 'moroto', 'mean': 3.5096724}, {'year_month': '2020_JJAS', 'bound_name': 'nakapiripirit', 'mean': 4.058996}, {'year_month': '2020_JJAS', 'bound_name': 'kaabong', 'mean': 3.3012788}, {'year_month': '2020_JJAS', 'bound_name': 'karenga', 'mean': 3.5930123}, {'year_month': '2020_JJAS', 'bound_name': 'amudat', 'mean': 4.715296}, {'year_month': '2020_JJAS', 'bound_name': 'nab_nak_napak', 'mean': 3.560986}, {'year_month': '2020_JJAS', 'bound_name': 'abi_kar_kaboong', 'mean': 3.1308608}, {'year_month': '2020_JJAS', 'bound_name': 'mor_kot_amudat', 'mean': 3.4663785}, {'year_month': '2020_JJAS', 'bound_name': 'karamoja', 'mean': 3.3748093}]
   year_month       bound_na

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2021_JJAS', 'bound_name': 'abim', 'mean': 3.1877644}, {'year_month': '2021_JJAS', 'bound_name': 'napak', 'mean': 4.330523}, {'year_month': '2021_JJAS', 'bound_name': 'nabilatuk', 'mean': 4.611858}, {'year_month': '2021_JJAS', 'bound_name': 'kotido', 'mean': 3.8148193}, {'year_month': '2021_JJAS', 'bound_name': 'moroto', 'mean': 4.74569}, {'year_month': '2021_JJAS', 'bound_name': 'nakapiripirit', 'mean': 5.342405}, {'year_month': '2021_JJAS', 'bound_name': 'kaabong', 'mean': 4.2743354}, {'year_month': '2021_JJAS', 'bound_name': 'karenga', 'mean': 4.857453}, {'year_month': '2021_JJAS', 'bound_name': 'amudat', 'mean': 5.9569063}, {'year_month': '2021_JJAS', 'bound_name': 'nab_nak_napak', 'mean': 4.670034}, {'year_month': '2021_JJAS', 'bound_name': 'abi_kar_kaboong', 'mean': 4.112581}, {'year_month': '2021_JJAS', 'bound_name': 'mor_kot_amudat', 'mean': 4.5866365}, {'year_month': '2021_JJAS', 'bound_name': 'karamoja', 'mean': 4.441263}]
   year_month       bound_name    

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]


In [33]:
folder_list=['IGAD_Standardized_Precipitation_Index_SPI1','IGAD_Standardized_Precipitation_Index_SPI3',
            'IGAD_Standardized_Precipitation_Index_SPI6','IGAD_Standardized_Precipitation_Index_SPI9',
            'IGAD_Standardized_Precipitation_Index_SPI12','IGAD_Standardized_Precipitation_Index_SPI24',
            'IGAD_Standardized_Precipitation_Index_SPI48']

var_list=['spg01','spg03','spg06','spg09','spg12','spg24','spg48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for MAM
start_month_index=2
end_month_index=4
    
for folder_str,var_str in zip(folder_list,var_list):
    mdb1=make_csv_season(geo_db2,admin_bound_list,folder_str,var_str,start_month_index,end_month_index)

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2000_MAM', 'bound_name': 'abim', 'mean': -1.0173641}, {'year_month': '2000_MAM', 'bound_name': 'napak', 'mean': -0.90197945}, {'year_month': '2000_MAM', 'bound_name': 'nabilatuk', 'mean': -0.96031964}, {'year_month': '2000_MAM', 'bound_name': 'kotido', 'mean': -0.7701611}, {'year_month': '2000_MAM', 'bound_name': 'moroto', 'mean': -1.0824914}, {'year_month': '2000_MAM', 'bound_name': 'nakapiripirit', 'mean': -0.90704435}, {'year_month': '2000_MAM', 'bound_name': 'kaabong', 'mean': -1.1770197}, {'year_month': '2000_MAM', 'bound_name': 'karenga', 'mean': -0.68138885}, {'year_month': '2000_MAM', 'bound_name': 'amudat', 'mean': -1.255953}, {'year_month': '2000_MAM', 'bound_name': 'nab_nak_napak', 'mean': -0.9158216}, {'year_month': '2000_MAM', 'bound_name': 'abi_kar_kaboong', 'mean': -1.0088159}, {'year_month': '2000_MAM', 'bound_name': 'mor_kot_amudat', 'mean': -0.98610973}, {'year_month': '2000_MAM', 'bound_name': 'karamoja', 'mean': -0.9719828}]
   year_month       b

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2001_MAM', 'bound_name': 'abim', 'mean': 0.7985218}, {'year_month': '2001_MAM', 'bound_name': 'napak', 'mean': 0.5663151}, {'year_month': '2001_MAM', 'bound_name': 'nabilatuk', 'mean': 0.601041}, {'year_month': '2001_MAM', 'bound_name': 'kotido', 'mean': 0.6448263}, {'year_month': '2001_MAM', 'bound_name': 'moroto', 'mean': 0.61915517}, {'year_month': '2001_MAM', 'bound_name': 'nakapiripirit', 'mean': 0.3776012}, {'year_month': '2001_MAM', 'bound_name': 'kaabong', 'mean': 0.5610784}, {'year_month': '2001_MAM', 'bound_name': 'karenga', 'mean': 0.82720983}, {'year_month': '2001_MAM', 'bound_name': 'amudat', 'mean': 0.33673584}, {'year_month': '2001_MAM', 'bound_name': 'nab_nak_napak', 'mean': 0.5215944}, {'year_month': '2001_MAM', 'bound_name': 'abi_kar_kaboong', 'mean': 0.69421166}, {'year_month': '2001_MAM', 'bound_name': 'mor_kot_amudat', 'mean': 0.5775558}, {'year_month': '2001_MAM', 'bound_name': 'karamoja', 'mean': 0.6020551}]
   year_month       bound_name     

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

...

[]
[{'year_month': '2019_MAM', 'bound_name': 'abim', 'mean': 1.484413}, {'year_month': '2019_MAM', 'bound_name': 'napak', 'mean': 2.024154}, {'year_month': '2019_MAM', 'bound_name': 'nabilatuk', 'mean': 1.8831805}, {'year_month': '2019_MAM', 'bound_name': 'kotido', 'mean': 1.257055}, {'year_month': '2019_MAM', 'bound_name': 'moroto', 'mean': 1.4364828}, {'year_month': '2019_MAM', 'bound_name': 'nakapiripirit', 'mean': 2.2464874}, {'year_month': '2019_MAM', 'bound_name': 'kaabong', 'mean': 1.3716372}, {'year_month': '2019_MAM', 'bound_name': 'karenga', 'mean': 1.594074}, {'year_month': '2019_MAM', 'bound_name': 'amudat', 'mean': 2.593481}, {'year_month': '2019_MAM', 'bound_name': 'nab_nak_napak', 'mean': 2.0555}, {'year_month': '2019_MAM', 'bound_name': 'abi_kar_kaboong', 'mean': 1.4587317}, {'year_month': '2019_MAM', 'bound_name': 'mor_kot_amudat', 'mean': 1.5763729}, {'year_month': '2019_MAM', 'bound_name': 'karamoja', 'mean': 1.6861385}]
   year_month       bound_name      mean
0    

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2020_MAM', 'bound_name': 'abim', 'mean': 2.2126195}, {'year_month': '2020_MAM', 'bound_name': 'napak', 'mean': 2.969993}, {'year_month': '2020_MAM', 'bound_name': 'nabilatuk', 'mean': 2.9830983}, {'year_month': '2020_MAM', 'bound_name': 'kotido', 'mean': 2.2769957}, {'year_month': '2020_MAM', 'bound_name': 'moroto', 'mean': 2.9697242}, {'year_month': '2020_MAM', 'bound_name': 'nakapiripirit', 'mean': 3.5449367}, {'year_month': '2020_MAM', 'bound_name': 'kaabong', 'mean': 2.58732}, {'year_month': '2020_MAM', 'bound_name': 'karenga', 'mean': 2.6186543}, {'year_month': '2020_MAM', 'bound_name': 'amudat', 'mean': 4.189094}, {'year_month': '2020_MAM', 'bound_name': 'nab_nak_napak', 'mean': 3.131601}, {'year_month': '2020_MAM', 'bound_name': 'abi_kar_kaboong', 'mean': 2.489362}, {'year_month': '2020_MAM', 'bound_name': 'mor_kot_amudat', 'mean': 2.9100888}, {'year_month': '2020_MAM', 'bound_name': 'karamoja', 'mean': 2.8279927}]
   year_month       bound_name      mean
0  

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2021_MAM', 'bound_name': 'abim', 'mean': 3.460913}, {'year_month': '2021_MAM', 'bound_name': 'napak', 'mean': 4.357305}, {'year_month': '2021_MAM', 'bound_name': 'nabilatuk', 'mean': 4.4885006}, {'year_month': '2021_MAM', 'bound_name': 'kotido', 'mean': 3.7101693}, {'year_month': '2021_MAM', 'bound_name': 'moroto', 'mean': 4.368142}, {'year_month': '2021_MAM', 'bound_name': 'nakapiripirit', 'mean': 5.2801647}, {'year_month': '2021_MAM', 'bound_name': 'kaabong', 'mean': 4.078611}, {'year_month': '2021_MAM', 'bound_name': 'karenga', 'mean': 4.270574}, {'year_month': '2021_MAM', 'bound_name': 'amudat', 'mean': 5.9467645}, {'year_month': '2021_MAM', 'bound_name': 'nab_nak_napak', 'mean': 4.640203}, {'year_month': '2021_MAM', 'bound_name': 'abi_kar_kaboong', 'mean': 3.9519875}, {'year_month': '2021_MAM', 'bound_name': 'mor_kot_amudat', 'mean': 4.389139}, {'year_month': '2021_MAM', 'bound_name': 'karamoja', 'mean': 4.3104887}]
   year_month       bound_name      mean
0   

In [34]:
folder_list=['IGAD_Standardized_Precipitation_Index_SPI1','IGAD_Standardized_Precipitation_Index_SPI3',
            'IGAD_Standardized_Precipitation_Index_SPI6','IGAD_Standardized_Precipitation_Index_SPI9',
            'IGAD_Standardized_Precipitation_Index_SPI12','IGAD_Standardized_Precipitation_Index_SPI24',
            'IGAD_Standardized_Precipitation_Index_SPI48']

var_list=['spg01','spg03','spg06','spg09','spg12','spg24','spg48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for OND
start_month_index=9
end_month_index=11
    
for folder_str,var_str in zip(folder_list,var_list):
    mdb1=make_csv_season(geo_db2,admin_bound_list,folder_str,var_str,start_month_index,end_month_index)

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2000_OND', 'bound_name': 'abim', 'mean': 0.56602174}, {'year_month': '2000_OND', 'bound_name': 'napak', 'mean': 0.7012774}, {'year_month': '2000_OND', 'bound_name': 'nabilatuk', 'mean': 0.62853277}, {'year_month': '2000_OND', 'bound_name': 'kotido', 'mean': 0.645572}, {'year_month': '2000_OND', 'bound_name': 'moroto', 'mean': 0.63843536}, {'year_month': '2000_OND', 'bound_name': 'nakapiripirit', 'mean': 0.5327152}, {'year_month': '2000_OND', 'bound_name': 'kaabong', 'mean': 0.5241144}, {'year_month': '2000_OND', 'bound_name': 'karenga', 'mean': 0.42306176}, {'year_month': '2000_OND', 'bound_name': 'amudat', 'mean': 0.5130848}, {'year_month': '2000_OND', 'bound_name': 'nab_nak_napak', 'mean': 0.63920105}, {'year_month': '2000_OND', 'bound_name': 'abi_kar_kaboong', 'mean': 0.51083285}, {'year_month': '2000_OND', 'bound_name': 'mor_kot_amudat', 'mean': 0.6182213}, {'year_month': '2000_OND', 'bound_name': 'karamoja', 'mean': 0.5859539}]
   year_month       bound_name   

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2001_OND', 'bound_name': 'abim', 'mean': 1.1044837}, {'year_month': '2001_OND', 'bound_name': 'napak', 'mean': 1.0357945}, {'year_month': '2001_OND', 'bound_name': 'nabilatuk', 'mean': 0.8613032}, {'year_month': '2001_OND', 'bound_name': 'kotido', 'mean': 1.1496907}, {'year_month': '2001_OND', 'bound_name': 'moroto', 'mean': 0.888444}, {'year_month': '2001_OND', 'bound_name': 'nakapiripirit', 'mean': 0.99867713}, {'year_month': '2001_OND', 'bound_name': 'kaabong', 'mean': 0.9713039}, {'year_month': '2001_OND', 'bound_name': 'karenga', 'mean': 1.0699074}, {'year_month': '2001_OND', 'bound_name': 'amudat', 'mean': 0.92353785}, {'year_month': '2001_OND', 'bound_name': 'nab_nak_napak', 'mean': 0.98832524}, {'year_month': '2001_OND', 'bound_name': 'abi_kar_kaboong', 'mean': 1.0333881}, {'year_month': '2001_OND', 'bound_name': 'mor_kot_amudat', 'mean': 1.0023364}, {'year_month': '2001_OND', 'bound_name': 'karamoja', 'mean': 1.0091391}]
   year_month       bound_name      

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

...

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2019_OND', 'bound_name': 'abim', 'mean': 1.4889566}, {'year_month': '2019_OND', 'bound_name': 'napak', 'mean': 2.3692944}, {'year_month': '2019_OND', 'bound_name': 'nabilatuk', 'mean': 2.413836}, {'year_month': '2019_OND', 'bound_name': 'kotido', 'mean': 1.7039449}, {'year_month': '2019_OND', 'bound_name': 'moroto', 'mean': 2.3761768}, {'year_month': '2019_OND', 'bound_name': 'nakapiripirit', 'mean': 2.908095}, {'year_month': '2019_OND', 'bound_name': 'kaabong', 'mean': 2.1192548}, {'year_month': '2019_OND', 'bound_name': 'karenga', 'mean': 2.180926}, {'year_month': '2019_OND', 'bound_name': 'amudat', 'mean': 3.4394057}, {'year_month': '2019_OND', 'bound_name': 'nab_nak_napak', 'mean': 2.5276241}, {'year_month': '2019_OND', 'bound_name': 'abi_kar_kaboong', 'mean': 1.9567025}, {'year_month': '2019_OND', 'bound_name': 'mor_kot_amudat', 'mean': 2.296134}, {'year_month': '2019_OND', 'bound_name': 'karamoja', 'mean': 2.246692}]
   year_month       bound_name      mean
0 

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

[]
[{'year_month': '2020_OND', 'bound_name': 'abim', 'mean': 3.0687554}, {'year_month': '2020_OND', 'bound_name': 'napak', 'mean': 3.9143183}, {'year_month': '2020_OND', 'bound_name': 'nabilatuk', 'mean': 4.030598}, {'year_month': '2020_OND', 'bound_name': 'kotido', 'mean': 3.4486694}, {'year_month': '2020_OND', 'bound_name': 'moroto', 'mean': 4.2100773}, {'year_month': '2020_OND', 'bound_name': 'nakapiripirit', 'mean': 4.7747912}, {'year_month': '2020_OND', 'bound_name': 'kaabong', 'mean': 3.9007876}, {'year_month': '2020_OND', 'bound_name': 'karenga', 'mean': 4.07221}, {'year_month': '2020_OND', 'bound_name': 'amudat', 'mean': 5.5623493}, {'year_month': '2020_OND', 'bound_name': 'nab_nak_napak', 'mean': 4.176802}, {'year_month': '2020_OND', 'bound_name': 'abi_kar_kaboong', 'mean': 3.7085736}, {'year_month': '2020_OND', 'bound_name': 'mor_kot_amudat', 'mean': 4.146747}, {'year_month': '2020_OND', 'bound_name': 'karamoja', 'mean': 3.997416}]
   year_month       bound_name      mean
0  

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/xarray/coding/

# SPEI seasonal

## very slow

In [4]:

import glob
import geopandas as gp
import pandas as pd
import numpy as np

import xarray as xr
import rioxarray

shpfiles=glob.glob('/home/data_folder/karamoja_shapefiles/*/*.shp')

dist_shp=gp.read_file(shpfiles[0])
#dist_shp.plot()
#dist_shp.info()

kara_shp=gp.read_file(shpfiles[1])
#kara_shp.plot()

lh_shp=gp.read_file(shpfiles[2])
#lh_shp.plot()

sc_shp=gp.read_file(shpfiles[3])
#sc_shp.plot()

dist_shp1=dist_shp[['admin2Name','geometry']]
dist_shp1.columns=['admin_name','geometry']
lh_shp1=lh_shp[['livelihood','geometry']]
lh_shp1.columns=['admin_name','geometry']
kara_shp1=kara_shp[['geometry']]
kara_shp1['admin_name']='karamoja'
#kara_shp1.columns=['admin_name','geometry']

geo_db=pd.concat([dist_shp1,lh_shp1,kara_shp1])
geo_db1=geo_db.reset_index()
geo_db2=geo_db1[['admin_name','geometry']]
geo_db2['admin_name1']=geo_db2['admin_name'].str.lower()
admin_bound_list=geo_db2['admin_name1'].tolist()


def region_shp_spei_averager_season(single_row_shp,xds,var_str,start_month_index,end_month_index):
    clipped = xds.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)
    da1=clipped[var_str]
    aa=da1.values
    aa1=aa[start_month_index:end_month_index,:,:]
    aa2=aa1.ravel()
    aa3 = aa2[(aa2 >=-9000)]
    cont_mean=np.mean(aa3)
    return cont_mean



def make_spei_csv_season(geo_db2,admin_bound_list,ncfilename,var_str,start_month_index,end_month_index):
    ncfile_path=f'/home/data_folder/ceda_spei/{ncfilename}.nc'
    month_index_str=get_month_index_str(start_month_index,end_month_index)
    #xds = rioxarray.open_rasterio(ncfile_path)
    xds=xr.open_dataset(ncfile_path)
    xds.rio.write_crs("epsg:4326", inplace=True)
    year_list=np.arange(1981,2016+1,1)
    mdb_cont=[]
    for year_int in year_list:
        #xds_year=xds.loc[dict(time=slice(str(year_int)))]
        xds_year=xds.sel(time=slice(str(year_int),str(year_int)))
        db_cont=[]
        year_cont=[]
        year_str=str(year_int)
        for ab_str in admin_bound_list:
            geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
            single_row_shp=geo_db3        
            cont_mean=region_shp_spei_averager_season(single_row_shp,xds_year,var_str,start_month_index,end_month_index)
            year_month_index_str=f'{year_str}_{month_index_str}'
            ab_dict={}
            ab_dict['year_month']=year_month_index_str
            ab_dict['bound_name']=ab_str
            ab_dict['mean']=cont_mean
            db_cont.append(ab_dict)
        print(year_cont)
        print(db_cont)
        db=pd.DataFrame(db_cont)
        print(db)
        db1=pd.pivot_table(db, values='mean', index=['year_month'], columns=['bound_name'])
        db2=db1.reset_index()
        mdb_cont.append(db2)
        db=[]
        db1=[]
        db2=[]
    mdb1=pd.concat(mdb_cont,axis=0)
    mdb1.to_csv(f'/home/data_folder/spei_seasonal/{ncfilename}_{month_index_str}.csv',index=False)
    mdb1=[]
    #return db

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:

ncfile_list=['spei01','spei03','spei06','spei09','spei12','spei24','spei48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for mar to october
start_month_index=2
end_month_index=9

var_str='spei'

for ncfilename in ncfile_list:
    make_spei_csv_season(geo_db2,admin_bound_list,ncfilename,var_str,start_month_index,end_month_index)


[]
[{'year_month': '1981_MAMJJASO', 'bound_name': 'abim', 'mean': 0.2655135}, {'year_month': '1981_MAMJJASO', 'bound_name': 'napak', 'mean': 0.18792722}, {'year_month': '1981_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.12997538}, {'year_month': '1981_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.3579538}, {'year_month': '1981_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.2577894}, {'year_month': '1981_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.15465875}, {'year_month': '1981_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.3404912}, {'year_month': '1981_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.29594105}, {'year_month': '1981_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.13698271}, {'year_month': '1981_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.16637734}, {'year_month': '1981_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.30826268}, {'year_month': '1981_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.27666268}, {'year_month': '1981_MAMJJASO', 'bound_name

[]
[{'year_month': '1986_MAMJJASO', 'bound_name': 'abim', 'mean': -0.69649446}, {'year_month': '1986_MAMJJASO', 'bound_name': 'napak', 'mean': -0.54947865}, {'year_month': '1986_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': -0.5960866}, {'year_month': '1986_MAMJJASO', 'bound_name': 'kotido', 'mean': -0.36746418}, {'year_month': '1986_MAMJJASO', 'bound_name': 'moroto', 'mean': -0.24789767}, {'year_month': '1986_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': -0.49478194}, {'year_month': '1986_MAMJJASO', 'bound_name': 'kaabong', 'mean': -0.14365949}, {'year_month': '1986_MAMJJASO', 'bound_name': 'karenga', 'mean': -0.22974214}, {'year_month': '1986_MAMJJASO', 'bound_name': 'amudat', 'mean': -0.32365015}, {'year_month': '1986_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': -0.544311}, {'year_month': '1986_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': -0.32106286}, {'year_month': '1986_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': -0.31104654}, {'year_month': '1986_MAMJJASO'

[]
[{'year_month': '1991_MAMJJASO', 'bound_name': 'abim', 'mean': 0.4891699}, {'year_month': '1991_MAMJJASO', 'bound_name': 'napak', 'mean': 0.54662824}, {'year_month': '1991_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.4794726}, {'year_month': '1991_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.3384259}, {'year_month': '1991_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.28504556}, {'year_month': '1991_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.5503246}, {'year_month': '1991_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.3704254}, {'year_month': '1991_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.26475328}, {'year_month': '1991_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.4483806}, {'year_month': '1991_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.5333259}, {'year_month': '1991_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.37768015}, {'year_month': '1991_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.33715576}, {'year_month': '1991_MAMJJASO', 'bound_name': 

[]
[{'year_month': '1996_MAMJJASO', 'bound_name': 'abim', 'mean': 0.44681025}, {'year_month': '1996_MAMJJASO', 'bound_name': 'napak', 'mean': 0.40826935}, {'year_month': '1996_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.32156414}, {'year_month': '1996_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.38226014}, {'year_month': '1996_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.4065627}, {'year_month': '1996_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.45139185}, {'year_month': '1996_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.27848002}, {'year_month': '1996_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.19406685}, {'year_month': '1996_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.42889893}, {'year_month': '1996_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.40168777}, {'year_month': '1996_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.30501044}, {'year_month': '1996_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.40069553}, {'year_month': '1996_MAMJJASO', 'bound_n

[]
[{'year_month': '2001_MAMJJASO', 'bound_name': 'abim', 'mean': 0.43175}, {'year_month': '2001_MAMJJASO', 'bound_name': 'napak', 'mean': 0.58520037}, {'year_month': '2001_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.695118}, {'year_month': '2001_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.3986913}, {'year_month': '2001_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.27818865}, {'year_month': '2001_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.6126967}, {'year_month': '2001_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.21646518}, {'year_month': '2001_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.3371628}, {'year_month': '2001_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.49251753}, {'year_month': '2001_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.6162395}, {'year_month': '2001_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.30720967}, {'year_month': '2001_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.36731318}, {'year_month': '2001_MAMJJASO', 'bound_name': 'k

[]
[{'year_month': '2006_MAMJJASO', 'bound_name': 'abim', 'mean': 0.45805278}, {'year_month': '2006_MAMJJASO', 'bound_name': 'napak', 'mean': 0.27056515}, {'year_month': '2006_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.27064732}, {'year_month': '2006_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.14771187}, {'year_month': '2006_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.12666859}, {'year_month': '2006_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.12651463}, {'year_month': '2006_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.094203465}, {'year_month': '2006_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.12830932}, {'year_month': '2006_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.08964772}, {'year_month': '2006_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.23079251}, {'year_month': '2006_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.20535901}, {'year_month': '2006_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.12848392}, {'year_month': '2006_MAMJJASO', 'bound

[]
[{'year_month': '2011_MAMJJASO', 'bound_name': 'abim', 'mean': 0.51733124}, {'year_month': '2011_MAMJJASO', 'bound_name': 'napak', 'mean': 0.54933584}, {'year_month': '2011_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.48250535}, {'year_month': '2011_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.5874995}, {'year_month': '2011_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.5279797}, {'year_month': '2011_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.3691593}, {'year_month': '2011_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.5832665}, {'year_month': '2011_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.69847685}, {'year_month': '2011_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.2906501}, {'year_month': '2011_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.48531273}, {'year_month': '2011_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.5932849}, {'year_month': '2011_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.50862384}, {'year_month': '2011_MAMJJASO', 'bound_name':

[]
[{'year_month': '2016_MAMJJASO', 'bound_name': 'abim', 'mean': -0.44254953}, {'year_month': '2016_MAMJJASO', 'bound_name': 'napak', 'mean': -0.15959015}, {'year_month': '2016_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': -0.08519414}, {'year_month': '2016_MAMJJASO', 'bound_name': 'kotido', 'mean': -0.25867933}, {'year_month': '2016_MAMJJASO', 'bound_name': 'moroto', 'mean': -0.1406546}, {'year_month': '2016_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': -0.03438338}, {'year_month': '2016_MAMJJASO', 'bound_name': 'kaabong', 'mean': -0.022939289}, {'year_month': '2016_MAMJJASO', 'bound_name': 'karenga', 'mean': -0.02362397}, {'year_month': '2016_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.016955497}, {'year_month': '2016_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': -0.10913736}, {'year_month': '2016_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': -0.14152704}, {'year_month': '2016_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': -0.1600747}, {'year_month': '2016_MAMJJAS

[]
[{'year_month': '1985_MAMJJASO', 'bound_name': 'abim', 'mean': 0.4791817}, {'year_month': '1985_MAMJJASO', 'bound_name': 'napak', 'mean': 0.25696918}, {'year_month': '1985_MAMJJASO', 'bound_name': 'nabilatuk', 'mean': 0.24781474}, {'year_month': '1985_MAMJJASO', 'bound_name': 'kotido', 'mean': 0.2691667}, {'year_month': '1985_MAMJJASO', 'bound_name': 'moroto', 'mean': 0.4357681}, {'year_month': '1985_MAMJJASO', 'bound_name': 'nakapiripirit', 'mean': 0.50937957}, {'year_month': '1985_MAMJJASO', 'bound_name': 'kaabong', 'mean': 0.5733007}, {'year_month': '1985_MAMJJASO', 'bound_name': 'karenga', 'mean': 0.7761854}, {'year_month': '1985_MAMJJASO', 'bound_name': 'amudat', 'mean': 0.37388647}, {'year_month': '1985_MAMJJASO', 'bound_name': 'nab_nak_napak', 'mean': 0.32473844}, {'year_month': '1985_MAMJJASO', 'bound_name': 'abi_kar_kaboong', 'mean': 0.59714943}, {'year_month': '1985_MAMJJASO', 'bound_name': 'mor_kot_amudat', 'mean': 0.35584238}, {'year_month': '1985_MAMJJASO', 'bound_name'

In [2]:
import glob
import geopandas as gp
import pandas as pd
import numpy as np

import xarray as xr
import rioxarray

shpfiles=glob.glob('/home/data_folder/karamoja_shapefiles/*/*.shp')

dist_shp=gp.read_file(shpfiles[0])
#dist_shp.plot()
#dist_shp.info()

kara_shp=gp.read_file(shpfiles[1])
#kara_shp.plot()

lh_shp=gp.read_file(shpfiles[2])
#lh_shp.plot()

sc_shp=gp.read_file(shpfiles[3])
#sc_shp.plot()

dist_shp1=dist_shp[['admin2Name','geometry']]
dist_shp1.columns=['admin_name','geometry']
lh_shp1=lh_shp[['livelihood','geometry']]
lh_shp1.columns=['admin_name','geometry']
kara_shp1=kara_shp[['geometry']]
kara_shp1['admin_name']='karamoja'
#kara_shp1.columns=['admin_name','geometry']

geo_db=pd.concat([dist_shp1,lh_shp1,kara_shp1])
geo_db1=geo_db.reset_index()
geo_db2=geo_db1[['admin_name','geometry']]
geo_db2['admin_name1']=geo_db2['admin_name'].str.lower()
admin_bound_list=geo_db2['admin_name1'].tolist()
#admin_bound_list



def get_month_index_str(start_month_index,end_month_index):
    month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
    m_ind_list=np.arange(start_month_index,end_month_index+1,1)
    m_cnt=[]
    for m_ind in m_ind_list:
        mstr=month_index_dict[m_ind]
        m_cnt.append(mstr)
    jmstr=''.join(m_cnt)
    return jmstr



def region_shp_spei_averager_season(single_row_shp,xds,var_str,start_month_index,end_month_index):
    month_index_str=get_month_index_str(start_month_index,end_month_index)
    year_list=np.arange(1981,2016+1,1)
    xds_clipped = xds.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)
    print(xds_clipped)
    db_cont=[]
    for year_int in year_list[0:3]:
        xds_year=xds_clipped.sel(time=slice(str(year_int),str(year_int)))
        da1=xds_year[var_str]
        aa=da1.values
        aa1=aa[start_month_index:end_month_index,:,:]
        aa2=aa1.ravel()
        aa3 = aa2[(aa2 >=-9000)]
        cont_mean=np.mean(aa3)
        year_str=str(year_int)
        year_month_index_str=f'{year_str}_{month_index_str}'
        ab_dict={}
        ab_dict['year_month']=year_month_index_str
        ab_dict['bound_name']=ab_str
        ab_dict['mean']=cont_mean
        db_cont.append(ab_dict)
    db=pd.DataFrame(db_cont)
    db1=pd.pivot_table(db, values='mean', index=['year_month'], columns=['bound_name'])
    db2=db1.reset_index()
    return db2,month_index_str



def make_spei_csv_season(geo_db2,admin_bound_list,ncfilename,var_str,start_month_index,end_month_index):
    ncfile_path=f'/home/data_folder/ceda_spei/{ncfilename}.nc'
    xds=xr.open_dataset(ncfile_path)
    xds.rio.write_crs("epsg:4326", inplace=True)
    mdb_cont=[]
    for ab_str in admin_bound_list[0:1]:
        geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
        single_row_shp=geo_db3        
        db2,month_index_str=region_shp_spei_averager_season(single_row_shp,xds,var_str,start_month_index,end_month_index)
        mdb_cont.append(db2)
        db2=[]
    mdb1=pd.concat(mdb_cont,axis=0)
    mdb1.to_csv(f'/home/data_folder/spei_seasonal/{ncfilename}_{month_index_str}.csv',index=False)
    mdb1=[]
    #return db

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
ncfile_list=['spei01','spei03','spei06','spei09','spei12','spei24','spei48']

#month_index_dict={0:'J',1:'F',2:'M',3:'A',4:'M',5:'J',6:'J',7:'A',8:'S',9:'O',10:'N',11:'D'}
#for mar to october
start_month_index=2
end_month_index=9

var_str='spei'

for ncfilename in ncfile_list[0:1]:
    make_spei_csv_season(geo_db2,admin_bound_list,ncfilename,var_str,start_month_index,end_month_index)

In [3]:
import xarray as xr

ncfile_path='/home/data_folder/ceda_spei/spei01.nc'

#xds = rioxarray.open_rasterio(ncfile_path)

xds=xr.open_dataset(ncfile_path)
xds.rio.write_crs("epsg:4326", inplace=True)
#xds_year=xds.loc[dict(time=slice(str(1991)))]

xds_year=xds.sel(time=slice('1991','1991'))

#xds_year['time']


ab_str=admin_bound_list[12]
print(ab_str)
geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
single_row_shp=geo_db3 

clipped = xds_year.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)
# year_xds=xds_year.indexes['time'].to_datetimeindex()
# year_str=str(year_xds[0].year)
# da=clipped.loc[dict(time=slice(year_str))]
# da1=da[var_str]
# aa=da1.values
# aa1=aa[start_month_index:end_month_index,:,:]
# aa2=aa1.ravel()
# aa3 = aa2[(aa2 >=-9000)]
# cont_mean=np.mean(aa3)
clipped

karamoja


<xarray.Dataset>
Dimensions:      (lat: 56, lon: 30, time: 12)
Coordinates:
  * lat          (lat) float32 4.175 4.125 4.075 4.025 ... 1.525 1.475 1.425
  * lon          (lon) float32 33.53 33.58 33.62 33.67 ... 34.88 34.92 34.97
  * time         (time) datetime64[ns] 1991-01-01 1991-02-01 ... 1991-12-01
    spatial_ref  int64 0
Data variables:
    spei         (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    title:                     High resolution Standardized Precipitation Eva...
    institution:               School of Geography and the Environment, Unive...
    version:                   1.0
    creators:                  Jian Peng, Simon Dadson, Feyera Hirpa, Ellen D...
    source:                    The dataset was produced using CHIRPS precipit...
    comment:                   The dataset was produced as part of the UK Spa...
    time_coverage:             1981.01-2016.12
    time_coverage_resolution:  monthly
    spatial_resolution:        5km

In [26]:
ncfile_path='/home/data_folder/icpac_data/data/IGAD_Standardized_Precipitation_Index_SPI1/CHIRPS/spg01_m_igad_200501_200512_m.nc'

xds = rioxarray.open_rasterio(ncfile_path)

#xds=xr.open_dataset(ncfile_path)
#xds.rio.write_crs("epsg:4326", inplace=True)
#xds_year=xds.loc[dict(time=slice(str(1991)))]

#xds_year=xds.sel(time=slice('1991','1991'))

#xds_year['time']


ab_str=admin_bound_list[12]
print(ab_str)
geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
single_row_shp=geo_db3 

clipped = xds.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)

clipped

karamoja


<xarray.Dataset>
Dimensions:             (time: 12, x: 30, y: 56)
Coordinates:
  * time                (time) object 2005-01-01 00:00:00 ... 2005-12-01 00:0...
  * x                   (x) float64 33.52 33.58 33.62 ... 34.88 34.92 34.98
  * y                   (y) float64 1.425 1.475 1.525 ... 4.075 4.125 4.175
    spatial_ref         int64 0
    latitude_longitude  int64 0
Data variables:
    spg01               (time, y, x) float32 -9.999e+03 ... -9.999e+03
Attributes: (12/28)
    lat#_FillValue:                                  nan
    latitude_longitude#crs_wkt:                      GEOGCS["WGS 84",DATUM["W...
    latitude_longitude#geographic_crs_name:          WGS 84
    latitude_longitude#GeoTransform:                 21.85 0.05 0.0 -11.75 0....
    latitude_longitude#grid_mapping_name:            latitude_longitude
    latitude_longitude#inverse_flattening:           298.257223563
    ...                                              ...
    spg01#grid_mapping:                              latitude_longitude
    spg01#units:                                     dimensionless
    spg01#_FillValue:                                -9999
    time#calendar:                                   proleptic_gregorian
    time#standard_name:                              time
    time#units:                                      days since 2005-01-01 00...

In [19]:
ncfile_path='/home/data_folder/icpac_data/data/IGAD_Standardized_Precipitation_Index_SPI1/CHIRPS/spg01_m_igad_200501_200512_m.nc'

#xds = rioxarray.open_rasterio(ncfile_path)

xds=xr.open_dataset(ncfile_path)
#xds.rio.write_crs("epsg:4326", inplace=True)
#xds_year=xds.loc[dict(time=slice(str(1991)))]

#xds_year=xds.sel(time=slice('1991','1991'))

#xds_year['time']


ab_str=admin_bound_list[6]
print(ab_str)
geo_db3=geo_db2[geo_db2['admin_name1']==ab_str]
single_row_shp=geo_db3 

#clipped = xds.rio.clip(single_row_shp.geometry.values, single_row_shp.crs)

#clipped
xds

kaabong


<xarray.Dataset>
Dimensions:             (lat: 698, lon: 591, time: 12)
Coordinates:
  * lat                 (lat) float64 -11.72 -11.68 -11.62 ... 23.02 23.07 23.12
  * lon                 (lon) float64 21.88 21.93 21.98 ... 51.27 51.33 51.38
  * time                (time) datetime64[ns] 2005-01-01 ... 2005-12-01
    latitude_longitude  int64 ...
Data variables:
    spg01               (time, lat, lon) float32 ...
    4326                float32 ...
Attributes:
    Source_Software:   netcdftonetcdfCF-1.6.py
    creator_name:      Created by dbinterface.py
    Conventions:       CF-1.6
    _CoordSysBuilder:  ucar.nc2.dataset.conv.CF1Convention
    date_created:      2021-12-05

In [89]:
xds_year1=xds_year.sel(x=slice(33.51,35.00))
xds_year2=xds_year1.sel(y=slice(1.36,4.23))
xds_year1

<xarray.DataArray 'spei' (time: 12, y: 1800, x: 1)>
array([[[9.96921e+36],
        [9.96921e+36],
        ...,
        [9.96921e+36],
        [9.96921e+36]],

       [[9.96921e+36],
        [9.96921e+36],
        ...,
        [9.96921e+36],
        [9.96921e+36]],

       ...,

       [[9.96921e+36],
        [9.96921e+36],
        ...,
        [9.96921e+36],
        [9.96921e+36]],

       [[9.96921e+36],
        [9.96921e+36],
        ...,
        [9.96921e+36],
        [9.96921e+36]]], dtype=float32)
Coordinates:
  * time         (time) object 1991-01-01 00:00:00 ... 1991-12-01 00:00:00
  * x            (x) float64 34.5
  * y            (y) float64 0.5 1.5 2.5 3.5 ... 1.798e+03 1.798e+03 1.8e+03
    spatial_ref  int64 0
Attributes:
    lat#standard_name:         latitude
    lat#units:                 degrees_north
    lon#standard_name:         longitude
    lon#units:                 degrees_east
    comment:                   The dataset was produced as part of the UK Spa...
    creators:                  Jian Peng, Simon Dadson, Feyera Hirpa, Ellen D...
    institution:               School of Geography and the Environment, Unive...
    source:                    The dataset was produced using CHIRPS precipit...
    spatial_resolution:        5km
    time_coverage:             1981.01-2016.12
    time_coverage_resolution:  monthly
    title:                     High resolution Standardized Precipitation Eva...
    version:                   1.0
    NETCDF_DIM_EXTRA:          {time}
    NETCDF_DIM_time_DEF:       [432.   6.]
    NETCDF_DIM_time_VALUES:    [    0.    31.    59.    90.   120.   151.   1...
    coordinates:               lat lon
    long_name:                 Standardized Precipitation Evapotranspiration ...
    units:                     ('z value', 'z value', 'z value', 'z value', '...
    _FillValue:                9.96921e+36
    scale_factor:              1.0
    add_offset:                0.0

In [88]:
clipped

<xarray.DataArray 'spei' (time: 12, y: 1, x: 1)>
array([[[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]],

       [[9.96921e+36]]], dtype=float32)
Coordinates:
  * time         (time) object 1991-01-01 00:00:00 ... 1991-12-01 00:00:00
  * x            (x) float64 34.5
  * y            (y) float64 2.5
    spatial_ref  int64 0
Attributes:
    lat#standard_name:         latitude
    lat#units:                 degrees_north
    lon#standard_name:         longitude
    lon#units:                 degrees_east
    comment:                   The dataset was produced as part of the UK Spa...
    creators:                  Jian Peng, Simon Dadson, Feyera Hirpa, Ellen D...
    institution:               School of Geography and the Environment, Unive...
    source:                    The dataset was produced using CHIRPS precipit...
    spatial_resolution:        5km
    time_coverage:             1981.01-2016.12
    time_coverage_resolution:  monthly
    title:                     High resolution Standardized Precipitation Eva...
    version:                   1.0
    NETCDF_DIM_EXTRA:          {time}
    NETCDF_DIM_time_DEF:       [432.   6.]
    NETCDF_DIM_time_VALUES:    [    0.    31.    59.    90.   120.   151.   1...
    coordinates:               lat lon
    long_name:                 Standardized Precipitation Evapotranspiration ...
    units:                     ('z value', 'z value', 'z value', 'z value', '...
    scale_factor:              1.0
    add_offset:                0.0
    _FillValue:                9.96921e+36

In [103]:
xds

<xarray.DataArray 'spei' (time: 432, y: 1800, x: 2080)>
[1617408000 values with dtype=float32]
Coordinates:
  * time         (time) object 1981-01-01 00:00:00 ... 2016-12-01 00:00:00
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 2.078e+03 2.078e+03 2.08e+03
  * y            (y) float64 0.5 1.5 2.5 3.5 ... 1.798e+03 1.798e+03 1.8e+03
    spatial_ref  int64 0
Attributes:
    lat#standard_name:         latitude
    lat#units:                 degrees_north
    lon#standard_name:         longitude
    lon#units:                 degrees_east
    comment:                   The dataset was produced as part of the UK Spa...
    creators:                  Jian Peng, Simon Dadson, Feyera Hirpa, Ellen D...
    institution:               School of Geography and the Environment, Unive...
    source:                    The dataset was produced using CHIRPS precipit...
    spatial_resolution:        5km
    time_coverage:             1981.01-2016.12
    time_coverage_resolution:  monthly
    title:                     High resolution Standardized Precipitation Eva...
    version:                   1.0
    NETCDF_DIM_EXTRA:          {time}
    NETCDF_DIM_time_DEF:       [432.   6.]
    NETCDF_DIM_time_VALUES:    [    0.    31.    59.    90.   120.   151.   1...
    coordinates:               lat lon
    long_name:                 Standardized Precipitation Evapotranspiration ...
    units:                     ('z value', 'z value', 'z value', 'z value', '...
    _FillValue:                9.96921e+36
    scale_factor:              1.0
    add_offset:                0.0

In [2]:
import cf_xarray as cfxr
import xarray as xr

ncfile_path='/home/data_folder/ceda_spei/spei01.nc'

#xds = rioxarray.open_rasterio(ncfile_path)

xds=xr.open_dataset(ncfile_path)

xds

<xarray.Dataset>
Dimensions:  (lat: 1800, lon: 2080, time: 432)
Coordinates:
  * lat      (lat) float32 47.97 47.92 47.88 47.83 ... -41.88 -41.92 -41.97
  * lon      (lon) float32 -27.98 -27.92 -27.88 -27.83 ... 75.88 75.93 75.97
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2016-12-01
Data variables:
    spei     (time, lat, lon) float32 ...
Attributes:
    title:                     High resolution Standardized Precipitation Eva...
    institution:               School of Geography and the Environment, Unive...
    version:                   1.0
    creators:                  Jian Peng, Simon Dadson, Feyera Hirpa, Ellen D...
    source:                    The dataset was produced using CHIRPS precipit...
    comment:                   The dataset was produced as part of the UK Spa...
    time_coverage:             1981.01-2016.12
    time_coverage_resolution:  monthly
    spatial_resolution:        5km

In [5]:
#"D:\docker_work\icpac_data\data\IGAD_Standardized_Precipitation_Index_SPI1\CHIRPS\spg01_m_igad_200501_200512_m.nc"
import rioxarray

ncfile_path='/home/data_folder/icpac_data/data/IGAD_Standardized_Precipitation_Index_SPI1/CHIRPS/spg01_m_igad_200501_200512_m.nc'
xds = rioxarray.open_rasterio(ncfile_path)

xds


<xarray.Dataset>
Dimensions:             (time: 12, x: 591, y: 698)
Coordinates:
  * time                (time) object 2005-01-01 00:00:00 ... 2005-12-01 00:0...
  * x                   (x) float64 21.88 21.93 21.98 ... 51.28 51.33 51.38
  * y                   (y) float64 -11.72 -11.67 -11.62 ... 23.02 23.08 23.12
    latitude_longitude  int64 0
Data variables:
    spg01               (time, y, x) float32 ...
Attributes: (12/28)
    lat#_FillValue:                                  nan
    latitude_longitude#crs_wkt:                      GEOGCS["WGS 84",DATUM["W...
    latitude_longitude#geographic_crs_name:          WGS 84
    latitude_longitude#GeoTransform:                 21.85 0.05 0.0 -11.75 0....
    latitude_longitude#grid_mapping_name:            latitude_longitude
    latitude_longitude#inverse_flattening:           298.257223563
    ...                                              ...
    spg01#grid_mapping:                              latitude_longitude
    spg01#units:                                     dimensionless
    spg01#_FillValue:                                -9999
    time#calendar:                                   proleptic_gregorian
    time#standard_name:                              time
    time#units:                                      days since 2005-01-01 00...

In [6]:
df=xr.open_dataset(ncfile_path)
df

<xarray.Dataset>
Dimensions:             (lat: 698, lon: 591, time: 12)
Coordinates:
  * lat                 (lat) float64 -11.72 -11.68 -11.62 ... 23.02 23.07 23.12
  * lon                 (lon) float64 21.88 21.93 21.98 ... 51.27 51.33 51.38
  * time                (time) datetime64[ns] 2005-01-01 ... 2005-12-01
    latitude_longitude  int64 ...
Data variables:
    spg01               (time, lat, lon) float32 ...
    4326                float32 ...
Attributes:
    Source_Software:   netcdftonetcdfCF-1.6.py
    creator_name:      Created by dbinterface.py
    Conventions:       CF-1.6
    _CoordSysBuilder:  ucar.nc2.dataset.conv.CF1Convention
    date_created:      2021-12-05